In [46]:
# minimal packages to import
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import *
import string
import re
from tqdm import tqdm
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from nltk.tokenize import word_tokenize
import regex as re
import requests

dir = r"C:\Users\ahmad\Downloads\your_file.csv"
df_courses = pd.read_csv(dir)

In [47]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load SnowballStemmer
snowstem = SnowballStemmer('english')

# Load stopwords to filter
lst_stopwords = set(stopwords.words('english'))

# Define the function to preprocess text
def preprocess_text(text):
    # Convert to lowercase this is because for some reason it wasn't elimination from stopword the word 'the' when starting with an uppercase
    text_lower = text.lower()

    # Tokenize, stem words, remove punctuation and remove stopwords
    stemmed_words = [snowstem.stem(word) for word in nltk.word_tokenize(text_lower) if not word in lst_stopwords and word.isalnum()]

    return stemmed_words

# Create column named 'descr_clean' and then apply preprocess_text to the 'Description' column in df_courses
df_courses['descr_clean'] = df_courses['Description'].apply(preprocess_text)

df_courses

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Course Name,University Name,Faculty Name,Description,Fees,Modality,Duration,City,Country,Link,administration,descr_clean
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,3D visualisation and animation play a role in ...,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[3d, visualis, anim, play, role, mani, area, p..."
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Businesses and governments rely on sound finan...,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[busi, govern, reli, sound, financi, knowledg,..."
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,"Our Accounting, Accountability & Financial Man...",Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[account, account, financi, manag, msc, cours,..."
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Embark on a professional accounting career wit...,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[embark, profession, account, career, academ, ..."
4,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Join us for an online session for prospective ...,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[join, us, onlin, session, prospect, student, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Materials Engineering,University of Padua,School of Engineering,The Master's degree Materials Engineering is a...,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,https://www.findamasters.com/masters-degrees/c...,On Campus,"[master, degre, materi, engin, interdisciplina..."
5996,Materials Engineering MSc,Swansea University,School of Engineering and Applied Sciences,The MSc in Materials Engineering provides you ...,Please visit our website for the Materials Eng...,MSc,1 year full-time; 2 years part-time; 3 years p...,Swansea,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[msc, materi, engin, provid, deep, understand,..."
5997,Materials Engineering MSc by Research,Swansea University,School of Engineering and Applied Sciences,Swansea is one of the UK’s leading centres for...,Please visit our website for the Materials Eng...,"MSc, Research Only",1 year full-time; 2 years part-time,Swansea,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[swansea, one, uk, lead, centr, materi, teach,..."
5998,"Materials Engineering with Industry, MSc",Swansea University,School of Engineering and Applied Sciences,Our MSc in Materials Engineering with Industry...,For current fees of the Materials Engineering ...,MSc,2 Years Full Time With a Year In Industry,Swansea,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[msc, materi, engin, industri, cours, open, in..."


In [48]:
# Get a dictionary with exchange rates where GBP is the base currency you want to use
# I saved the API key in another text file. There is no need to do it in this case but I thought it could be good practice

# Read API key from the file
with open(r"C:\Users\ahmad\Desktop\sap\adm\HW3\API KEY\API KEY.txt", 'r') as api_key_file:
    api_key = api_key_file.read().strip()

# URL with the API key
url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/GBP'

# Making our request for the conversion rates
response = requests.get(url)
data_exchange = response.json()

with open('data_exchange.json', 'w') as json_file:
    json.dump(data_exchange, json_file)

# Read the JSON data from the file
with open('data_exchange.json', 'r') as json_file:
    loaded_data = json.load(json_file)

# Convert the 'result' column to a Pandas DataFrame
df = pd.DataFrame(loaded_data)

# Extract only the 'conversion_rates' column
conversion_rates = df['conversion_rates']

In [49]:
# Defined a function to select the maximum fee from a list of fees
def max_value_fee(list_fees):
    filtered_fee_list = [int(fee) for fee in list_fees]
    if filtered_fee_list:
        max_fee = max(filtered_fee_list)
        max_fee = float(max_fee)
    else:
        # If the list is empty, set max_fee to None
        max_fee = None
    return max_fee

# Define a function to remove dots and commas from numbers in a string
def remove_dots_from_numbers(input_string):
    # Use regular expression to find numbers with dots or commas and remove the dots or commas
    result_string = re.sub(r'(\d)[.,](\d{3})', r'\1\2', input_string)
    return result_string

# Iterate over each row in df_courses
for index, row in df_courses.iterrows():
    # Extract the 'Fees' column as a string
    fees_string = str(row['Fees'])

    # Remove dots from numbers and specific year strings (These years tend to create problems)
    fees_string = remove_dots_from_numbers(fees_string)
    fees_string = fees_string.replace("2022", '').replace("2023", '').replace("2024", '')

    # Remove various punctuation, spaces, and special characters
    fees_string = fees_string.replace(".00 ", '').replace(".00€ ", '').replace(",00 ", '').replace(",00€ ", '').replace(".0 ", '').replace(",0 ", '').replace('.', '').replace(',', '').replace("'", '').replace(" ", '')

    # Use regular expression to find currency symbols or codes ISO 4217 for all currencies (Those are the once that are used in our conversion_rates variable)
    matches_cur = re.findall(r'HKD(?:s)?|HK(?:s)?|\p{Sc}|euro(?:s)?|dollar(?:s)?|pound(?:s)?|EUR(?:s)?|USD(?:s)?|CHF(?:s)?|SEK(?:s)?|ISK(?:s)?|RMB(?:s)?|QR(?:s)?|GBP(?:s)?|QAR(?:s)?|JPY(?:s)?', fees_string, flags=re.IGNORECASE)

    # Use regular expression to find fees (excluding specific years) in the cleaned string
    matches_fee = re.findall(r'(?!2021|2022|2023|2024)\d{4,}', fees_string)
    # Call the max_value_fee function to get the maximum fee
    fees_float = max_value_fee(matches_fee)

    # Check if currency matches and assign the corresponding ISO 4217 code for all currencies (Those are the once that are used in our conversion_rates variable)
    if matches_cur:
        currency = None
        if matches_cur[0] == 'euro' or matches_cur[0] == 'euros' or matches_cur[0] == '€' or matches_cur[0] == 'EURs' or matches_cur[0] == 'Eur' or matches_cur[0] == 'Euros' or matches_cur[0] == 'EUROS' or matches_cur[0] == 'eurs'or matches_cur[0] == 'Euro':
            currency = 'EUR'
        elif matches_cur[0] == 'dollar' or matches_cur[0] == 'dollars' or matches_cur[0] == '$':
            currency = 'USD'
        elif matches_cur[0] == 'pound' or matches_cur[0] == 'pounds' or matches_cur[0] == '£':
            currency = 'GBP'
        elif matches_cur[0] == 'RMB':
            currency = 'CNY'
        elif matches_cur[0] == 'QR':
            currency = 'QAR'
        elif matches_cur[0] == 'HK':
            currency = 'HKD'
        else:
            currency = matches_cur[0]
    else:
        currency = None
    # Initialize fees_pound as None
    fees_pound = None

    # Check if fees_float is not None and less than 100000 before rounding and conversion (fees (£) above 100000 are due to super specific errors, less than 10 which are more reasonable to deal in this way)
    if fees_float is not None and fees_float < 100000:
        fees_pound = round(fees_float / conversion_rates.get(currency, 1.0), 2)

    # Assign the calculated fees in pounds to the 'fees (£)' column
    df_courses.at[index, 'fees (£)'] = fees_pound

# Filter the df_cources to include only rows where 'fees (£)' is not null
filtered_df = df_courses[(df_courses['fees (£)'].notnull())]
filtered_df


,Course Name,University Name,Faculty Name,Description,Fees,Modality,Duration,City,Country,Link,administration,descr_clean,fees (£)
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Businesses and governments rely on sound finan...,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[busi, govern, reli, sound, financi, knowledg,...",34750.00
5,Advanced Chemical Engineering - MSc,University of Leeds,School of Chemical and Process Engineering,The Advanced Chemical Engineering MSc at Leeds...,"UK: £13,750 (Total) International: £31,000 (To...",MSc,1 year full time,Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[advanc, chemic, engin, msc, leed, build, core...",31000.00
7,Agricultural Sciences - MSc (Agriculture and F...,University of Helsinki,International Masters Degree Programmes,Goal of the pro­grammeWould you like to be inv...,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,https://www.findamasters.com/masters-degrees/c...,On Campus,"[goal, like, involv, find, solut, futur, chall...",13026.49
8,"Agricultural, Environmental and Resource Econo...",University of Helsinki,International Masters Degree Programmes,Goal of the pro­grammeAre you looking forward ...,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,https://www.findamasters.com/masters-degrees/c...,On Campus,"[goal, look, forward, futur, expert, agricultu...",13026.49
9,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Up to 7 million people are estimated to die ev...,"UK: £12,500 (Total) International: £28,750 (To...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[7, million, peopl, estim, die, everi, year, d...",28750.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Harbour.Space's FinTech Master programme is de...,"€29,900/year","MBA, MSc",1 Year,Barcelona,Spain,https://www.findamasters.com/masters-degrees/c...,On Campus,"[fintech, master, programm, design, prepar, gr...",25966.13
5984,Master's of Front-end Development,Harbour.Space University,Masters Programmes,Front-end Development at Harbour.Space Univers...,"€29,900/year",MSc,1 year,Barcelona,Spain,https://www.findamasters.com/masters-degrees/c...,On Campus,"[develop, univers, provid, uniqu, environ, stu...",25966.13
5992,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Register your interest in graduate study at UC...,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[regist, interest, graduat, studi, uclther, gr...",14100.00
5995,Materials Engineering,University of Padua,School of Engineering,The Master's degree Materials Engineering is a...,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,https://www.findamasters.com/masters-degrees/c...,On Campus,"[master, degre, materi, engin, interdisciplina...",2344.77


In [5]:
df_courses = pd.DataFrame(df_courses)

In [50]:
df_courses['Course Name'] = df_courses['Course Name'].str.replace('-', '')
df_courses['Course Name'] = df_courses['Course Name'].str.replace('msc', '')
df_courses['Course Name'] = df_courses['Course Name'].str.replace('Msc', '')
df_courses['coursename_clean'] = df_courses['Course Name'].str.lower().str.split()
df_courses['uni_name_clean'] = df_courses['University Name'].str.lower()
df_courses['city_lowered'] = df_courses['City'].str.lower()

#
df_courses

,Course Name,University Name,Faculty Name,Description,Fees,Modality,Duration,City,Country,Link,administration,descr_clean,fees (£),coursename_clean,uni_name_clean,city_lowered
0,3D Design for Virtual Environments MSc,Glasgow Caledonian University,School of Engineering and Built Environment,3D visualisation and animation play a role in ...,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[3d, visualis, anim, play, role, mani, area, p...",NaN,"[3d, design, for, virtual, environments, msc]",glasgow caledonian university,glasgow
1,Accounting and Finance MSc,University of Leeds,Leeds University Business School,Businesses and governments rely on sound finan...,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[busi, govern, reli, sound, financi, knowledg,...",34750.00,"[accounting, and, finance, msc]",university of leeds,leeds
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,"Our Accounting, Accountability & Financial Man...",Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[account, account, financi, manag, msc, cours,...",NaN,"[accounting,, accountability, &, financial, ma...",king’s college london,london
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Embark on a professional accounting career wit...,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[embark, profession, account, career, academ, ...",NaN,"[accounting,, financial, management, and, digi...",university of reading,reading
4,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Join us for an online session for prospective ...,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[join, us, onlin, session, prospect, student, ...",NaN,"[addictions, msc]",king’s college london,london
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Materials Engineering,University of Padua,School of Engineering,The Master's degree Materials Engineering is a...,Our tuition fees will not exceed 2700 euros pe...,MSc,2 years,Padua,Italy,https://www.findamasters.com/masters-degrees/c...,On Campus,"[master, degre, materi, engin, interdisciplina...",2344.77,"[materials, engineering]",university of padua,padua
5996,Materials Engineering MSc,Swansea University,School of Engineering and Applied Sciences,The MSc in Materials Engineering provides you ...,Please visit our website for the Materials Eng...,MSc,1 year full-time; 2 years part-time; 3 years p...,Swansea,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[msc, materi, engin, provid, deep, understand,...",NaN,"[materials, engineering, msc]",swansea university,swansea
5997,Materials Engineering MSc by Research,Swansea University,School of Engineering and Applied Sciences,Swansea is one of the UK’s leading centres for...,Please visit our website for the Materials Eng...,"MSc, Research Only",1 year full-time; 2 years part-time,Swansea,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Campus,"[swansea, one, uk, lead, centr, materi, teach,...",NaN,"[materials, engineering, msc, by, research]",swansea university,swansea
5998,"Materials Engineering with Industry, MSc",Swansea University,School of Engineering and Applied Sciences,Our MSc in Materials Engineering with Industry...,For current fees of the Materials Engineering ...,MSc,2 Years Full Time With a Year In Industry,Swansea,United Kingdom,https://www.findamasters.com/masters-degrees/c...,On Ca

In [51]:
all_words_set = set()

# Function to extract words from a course name and add them to the set
def extract_words(course_name):
    words = course_name.split()
    all_words_set.update(words)

# Apply the function to each row in the 'course name' column
df_courses['Course Name'].apply(extract_words)

# Display the set of all unique words
print(all_words_set)

{'Design', 'Precision', 'FIFA', '(Hydrographic', 'Animate', '(Palaeoanthropology)', '(Waste,', 'Management', 'South', 'Hydrology', 'Nurse', 'Practice)', 'Planning)', 'Extremist', 'Implant', 'English', 'year)', 'programme)', 'Campus,', 'Accreditation)', '(Parttime', 'PGCert/PGDip/MSc', 'Crime,', '(Grenoble)', 'Inflammation', '(Consumer', 'HEMS', 'Yacht', 'Manufacture', 'validation)', 'Biochemistry)', '(Cellular', 'Insurance', 'Migrations', 'Chemistry', 'Collaboration', 'Trading', 'Needs', 'Earthquake', 'Programmable', '(Health', 'Disorders', 'Adaptation', 'Haematology', '(Online)', 'Territories', 'Games', 'Chronic', 'Failure', '(LPC)', 'Clinical', 'Infection,', 'East', 'Engineers', 'Urbanism', 'MHS/MSc', 'Regulatory', 'Nutrition,', '(Policy', 'General', 'Renewable', 'Elite', '(Distance', 'Wilderness', 'online', '(Minimallyinvasive', 'Optical', 'Wound', 'Aeronautics,', 'Policy,', '(Infection', 'Additive', 'ENGINEERING', 'Microsystems', 'Administration', 'Apprenticeship', '(M.A.,', 'place

In [52]:
vocabulary = {}
unique_id = 1
for word in list(all_words_set):
  vocabulary[unique_id] = word.lower()
  unique_id += 1

In [33]:
# Loop through the keys of the vocabulary
inverted_index = dict()
for i in list(vocabulary.keys()):
    inverted_index[i] = []
    for index, row in df_courses.iterrows():
        if vocabulary[i] in row['coursename_clean']:
            inverted_index[i].append(index)

In [53]:

def search_documents(split_cn):
    result_indices = set()
    for term in split_cn:
        if term in vocabulary.values():
            term_id = [key for key, value in vocabulary.items() if value == term][0]
            result_indices.update(inverted_index[term_id])
    return result_indices


In [56]:
import pandas as pd

def search_eng():
    # Collect user inputs
    course_name = input('Enter Course Name (press enter to skip): ').lower()
    uni_name = input('Enter University Name (press enter to skip): ').lower()
    city = input('Enter City Name (press enter to skip): ').lower()
    online = input('Do you need an online modality? Reply with Y or N (press enter to skip): ').lower()

    # Default values for price range
    min_price = input('Enter Minimum Price (press enter to skip): ')
    max_price = input('Enter Maximum Price (press enter to skip): ')

    # Apply filters based on user inputs
    filtered_courses = df_courses.copy()

    if course_name:
        course_name_split = set(course_name.split())
        course_indices = search_documents(course_name_split)
        filtered_courses = filtered_courses.loc[list(course_indices)]

    if uni_name:
        uni_name_lower = uni_name.lower()
        filtered_courses = filtered_courses[filtered_courses['uni_name_clean'].str.contains(uni_name_lower)]

    if city:
        city_lower = city.lower()
        filtered_courses = filtered_courses[filtered_courses['city_lowered'].str.contains(city_lower)]

    # Filter based on Fees (£)
    filtered_courses['fees (£)'] = filtered_courses['fees (£)'].apply(lambda x: x if pd.notna(x) and str(x).replace('.', '', 1).isdigit() else 'Refer to university website for fees')

    # Filter based on price range
    if min_price and max_price:
        # Exclude courses with 'Refer to university website for fees' if user provides a fee range
        filtered_courses = filtered_courses[(filtered_courses['fees (£)'] != 'Refer to university website for fees') & (filtered_courses['fees (£)'] >= int(min_price)) & (filtered_courses['fees (£)'] <= int(max_price))]
    elif min_price:
        filtered_courses = filtered_courses[(filtered_courses['fees (£)'] != 'Refer to university website for fees') & (filtered_courses['fees (£)'] >= int(min_price))]
    elif max_price:
        filtered_courses = filtered_courses[(filtered_courses['fees (£)'] != 'Refer to university website for fees') & (filtered_courses['fees (£)'] <= int(max_price))]

    if online == 'y':
        filtered_courses = filtered_courses[filtered_courses['administration'].str.contains('online', case=False)]
    elif online == 'n':
        filtered_courses = filtered_courses[~filtered_courses['administration'].str.contains('online', case=False)]

    # Display the final result
    if not filtered_courses.empty:
        # Print only the required columns
        result_columns = ['Course Name', 'University Name', 'Link']
        print("Filtered courses:")
        print(filtered_courses[result_columns])
    else:
        print("No courses match the specified criteria.")

# Call the search engine function
search_eng()


Filtered courses:
                                   Course Name  \
512                Accounting and Finance  MSc   
1                  Accounting and Finance  MSc   
513                Accounting and Finance  MSc   
514                Accounting and Finance  MSc   
515                Accounting and Finance  MSc   
...                                        ...   
4603      International Accounting and Finance   
508   Accounting and Business Intelligence MSc   
509     Accounting and Business Management MSc   
510          Accounting and Data Analytics MSc   
511   Accounting and Finance  Fast track (MSc)   

                                 University Name  \
512                     University of Birmingham   
1                            University of Leeds   
513   University of the West of England, Bristol   
514                           Cardiff University   
515                            Bangor University   
...                                          ...   
4603             